## HTML to Beautiful Soup

In [16]:
from __future__ import print_function, division #acquiring html:
import requests
import re
url = 'https://www.nutritionvalue.org/Fish%2C_surimi_nutritional_value.html'

#response = requests.get(url)

In [2]:
response.status_code

200

In [5]:
print(response.text)


<!DOCTYPE html>
<html>
<head>
<!-- This document is protected by the copyright law. 
Modification, publishing and redistribution of this document are prohibited. -->
<meta name="Author" content="Vitalii Vanovschi">
<meta name="Description" content="Fish, surimi: nutritional value and analysis">
<meta name="KeyWords" content="Fish, surimi: nutritional value and analysis">

<title>Fish, surimi: nutritional value and analysis</title>
<link rel='stylesheet' href='https://code.jquery.com/ui/1.10.2/themes/ui-lightness/jquery-ui.css' type='text/css'>
<style type="text/css">
body {
    font-family: 'Arial';
}
table {
    text-align: center;
}
tr.header {
    height: 100px;
}
tr.spacer {
    height: 50px;
}
a.l {
    padding: 4px;
}
form {
    margin:0px;
}

/* Css for search results pages */
table.results {
  border-collapse: collapse;
  text-align:left;
}

div.results {
	padding: 10px 0;
}

table.results th {
  background-color: #DBEEE7;
  padding: 10px;  
  white-space:nowrap;
}
table.resul

In [3]:
page = response.text #turning text into soup object:

In [17]:
from bs4 import BeautifulSoup



# Pulling information out of HTML

In [18]:
def get_fish_name(soup): 
    title = soup.find("h1").text #find title of page for row label
    return title

In [ ]:
title = get_fish_name(soup)

In [98]:

selenium_regex = re.compile('Vitamin\sA') #now looking for one label
selenium_string = soup.find(text=selenium_regex)
print(selenium_string)

Vitamin A


In [101]:
se_val = selenium_string.findNext().findNext().text #now looking for the value after Selenium label
print(se_val) 

•


In [15]:
def get_nutrient_val(soup_object, nutrient): #scaling up, but only takes one nutrient
    '''Grab a value from nutritionvalue HTML
    
    Takes a string attribute on the page and
    returns the string in the next object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup_object.find(text=re.compile(nutrient))
    if not obj: 
        return '0' 
    next_val = obj.findNext()
    if next_val:
        return next_val.text 
    else:
        return '0' 

In [268]:
(get_nutrient_val(soup, 'Selenium')).encode("ascii", "ignore") #testing function 

b'28.1mcg'

In [213]:
type(get_nutrient_val(soup, 'Riboflavin'))

str

In [19]:
def all_nutrients(list_of_nutrients, soup_object): #wrapper function to take a list of nutrients and output a list
    list_of_nuts = list([(get_nutrient_val(soup_object, nutrient)) for nutrient in list_of_nutrients])
    return(list_of_nuts)

## List of Parameters to be Included

In [215]:
parameters_list = ['Selenium', 'Saturated\sfatty\sacids', 'Polyunsaturated\sfatty\sacids', 'Monounsaturated\sfatty\sacids', 'Potassium', 'Vitamin\sD', 'Vitamin\sA', 'Vitamin\sC', 'Calcium', 'Iron', 'Sodium', 'Protein']

In [359]:
nutrients = all_nutrients(parameters_list, soup) 

Problem: Everything after "Vitamin A" is giving me a percentage rather than a value. Proposed solution: create second scrape and put them in lists, divide out the first bit, smush the lists together, and then pull out everything that's not a number. <<<< solved:: don't need to change them over because that info is all you'll ever have on a nutrition label anyway, so I shouldn't even.

# Cleaning data

In [20]:
def clean_numbers(list_of_strings): #clean the numbers and turn them into floats
    cleaned_nums = []
    for s in list_of_strings:
        s1=s.replace("\xc2\xa0", " ").split(None, 1)[0]
        f = float(s1)
        cleaned_nums.append(f)
    return(cleaned_nums)

In [361]:
clean_nutrients = clean_numbers(nutrients)
clean_nutrients

[28.1, 0.191, 0.443, 0.149, 112.0, 0.0, 1.0, 0.0, 1.0, 1.0, 6.0, 30.0]

# Compiling final data object

In [21]:
def compile_tuple(list_of_cleaned_nums, fish_title): #compiles final tuple to be turned into a list with the others
    list_of_cleaned_nums.append(fish_title)
    fish_tup = tuple(list_of_cleaned_nums)
    return(fish_tup)

In [356]:
fish_tup = compile_tuple(clean_nutrients, title)

In [357]:
fish_tup

(28.1,
 0.191,
 0.443,
 0.149,
 112.0,
 0.0,
 1.0,
 0.0,
 1.0,
 1.0,
 6.0,
 30.0,
 'Fish, surimi')

# Scraping multiple files

First attempt:

In [319]:
url_wrapper = 'https://www.nutritionvalue.org/Figs%2C_stewed%2C_dried_nutritional_value.html'
response_wrapper = requests.get(url_wrapper)
page_wrapper = response_wrapper.text
soup_wrapper = BeautifulSoup(page_wrapper, "lxml")
print(soup_wrapper.prettify())

<!DOCTYPE html>
<html>
 <head>
  <!-- This document is protected by the copyright law. 
Modification, publishing and redistribution of this document are prohibited. -->
  <meta content="Vitalii Vanovschi" name="Author"/>
  <meta content="Figs, stewed, dried: nutritional value and analysis" name="Description"/>
  <meta content="Figs, stewed, dried: nutritional value and analysis" name="KeyWords"/>
  <title>
   Figs, stewed, dried: nutritional value and analysis
  </title>
  <link href="https://code.jquery.com/ui/1.10.2/themes/ui-lightness/jquery-ui.css" rel="stylesheet" type="text/css"/>
  <style type="text/css">
   body {
    font-family: 'Arial';
}
table {
    text-align: center;
}
tr.header {
    height: 100px;
}
tr.spacer {
    height: 50px;
}
a.l {
    padding: 4px;
}
form {
    margin:0px;
}

/* Css for search results pages */
table.results {
  border-collapse: collapse;
  text-align:left;
}

div.results {
	padding: 10px 0;
}

table.results th {
  background-color: #DBEEE7;
  padd

In [318]:
response_wrapper.status_code

403

The "parent" lists won't let you scrape them. New strategy: there's only 7 of them, grab the html manually and clean it to get list of urls.

### Initial test batch:

In [376]:
test_urls = ['https://www.nutritionvalue.org/Fish%2C_raw%2C_ling_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_cusk_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_carp_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_spot_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_scup_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_cisco_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_burbot_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_lingcod_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_haddock_nutritional_value.html']

In [22]:
def test_scrape(test_urls):
    list_of_test_tups = []
    for url in test_urls:
        response1 = requests.get(url)
        page1 = response1.text
        soup1 = BeautifulSoup(page1, "lxml")
        title1 = get_fish_name(soup1)
        parameters_list1 = ['Selenium', 'Saturated\sfatty\sacids', 'Polyunsaturated\sfatty\sacids', 'Monounsaturated\sfatty\sacids', 'Potassium', 'Vitamin\sD', 'Vitamin\sA', 'Vitamin\sC', 'Calcium', 'Iron', 'Sodium', 'Protein']
        nutrients1 = all_nutrients(parameters_list1, soup1)
        clean_nutrients1 = clean_numbers(nutrients1)
        fish_tup1 = compile_tuple(clean_nutrients1, title1)
        list_of_test_tups.append(fish_tup1)
    return list_of_test_tups

In [379]:
#function_test_results = test_scrape(test_urls)

In [380]:
#function_test_results

[(36.5,
  0.12,
  0.22,
  0.09,
  379.0,
  0.0,
  2.0,
  0.0,
  3.0,
  4.0,
  6.0,
  38.0,
  'Fish, raw, ling'),
 (36.5,
  0.13,
  0.28,
  0.09,
  392.0,
  0.0,
  1.0,
  0.0,
  1.0,
  5.0,
  1.0,
  38.0,
  'Fish, raw, cusk'),
 (12.6,
  1.083,
  1.431,
  2.328,
  333.0,
  988.0,
  1.0,
  3.0,
  4.0,
  7.0,
  2.0,
  36.0,
  'Fish, raw, carp'),
 (36.5,
  1.45,
  1.09,
  1.33,
  496.0,
  0.0,
  2.0,
  0.0,
  1.0,
  2.0,
  1.0,
  38.0,
  'Fish, raw, spot'),
 (36.5,
  0.64,
  1.03,
  0.56,
  287.0,
  47.0,
  2.0,
  0.0,
  4.0,
  3.0,
  2.0,
  38.0,
  'Fish, raw, scup'),
 (12.6,
  0.421,
  0.613,
  0.469,
  354.0,
  0.0,
  2.0,
  0.0,
  1.0,
  2.0,
  2.0,
  38.0,
  'Fish, raw, cisco'),
 (12.6,
  0.163,
  0.297,
  0.133,
  404.0,
  0.0,
  0.0,
  0.0,
  5.0,
  5.0,
  4.0,
  38.0,
  'Fish, raw, burbot'),
 (36.5,
  0.197,
  0.3,
  0.35,
  437.0,
  0.0,
  1.0,
  0.0,
  1.0,
  2.0,
  3.0,
  36.0,
  'Fish, raw, lingcod'),
 (25.9,
  0.091,
  0.166,
  0.061,
  286.0,
  18.0,
  1.0,
  0.0,
  1.0,
  1.0

### Test Batch to Data Frame

In [23]:
labels = ['Selenium', 'SFA', 'PUFA', 'MUFA', 'Potassium', 'Vit_D', 'Vit_A', 'Vit_C', 'Calcium', 'Iron', 'Sodium', 'Protein', 'Fish']

In [30]:
import pandas as pd
df_test = pd.DataFrame.from_records(function_test_results, columns = labels)

In [32]:
df_test #it does look weird that there are all those 36.5 and 12.6. It's in the initial data set, though.

,Selenium,SFA,PUFA,MUFA,Potassium,Vit_D,Vit_A,Vit_C,Calcium,Iron,Sodium,Protein,Fish
0,28.1,0.191,0.443,0.149,112.0,0.0,1.0,0.0,1.0,1.0,6.0,30.0,"Fish, surimi"
1,14.1,0.076,0.102,0.053,285.0,8.0,1.0,0.0,2.0,8.0,3.0,32.0,"Frog legs, raw"
2,36.5,0.120,0.220,0.090,379.0,0.0,2.0,0.0,3.0,4.0,6.0,38.0,"Fish, raw, ling"
3,36.5,0.130,0.280,0.090,392.0,0.0,1.0,0.0,1.0,5.0,1.0,38.0,"Fish, raw, cusk"
4,12.6,1.083,1.431,2.328,333.0,988.0,1.0,3.0,4.0,7.0,2.0,36.0,"Fish, raw, carp"
5,36.5,1.450,1.090,1.330,496.0,0.0,2.0,0.0,1.0,2.0,1.0,38.0,"Fish, raw, spot"
6,36.5,0.640,1.030,0.560,287.0,47.0,2.0,0.0,4.0,3.0,2.0,38.0,"Fish, raw, scup"
7,41.2,1.544,4.122,2.887,178.0,0.0,2.0,4.0,2.0,6.0,17.0,32.0,"Fish, tuna salad"
8,12.6,0.421,0.613,0.469,354.0,0.0,2.0,0.0,1.0,2.0,2.0,38.0,"Fish, raw, cisco"
9,12.6,0.163,0.297,0.133,404.0,0.0,0.0,0.0,5.0,5.0,4.0,38.0,"Fish, raw, burbot"


In [34]:
df_test.to_csv('sec1_fishes.csv', index=False)

## Generating full URL list from manual copy of HTML (since the parent list page wouldn't let me scrape)

In [24]:
page = '<tr><td><table class="full_width results zero"><tbody><tr class="results"><th class="results">Food Name&nbsp;&nbsp;<span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;descending&quot;)" title="Click to sort in descending order.">▲</span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;ascending&quot;)" title="Click to sort in ascending order.">▼</span></span></th><th class="results">Action</th></tr><tr class="results"><td class="results left"><a href="/Fish%2C_surimi_nutritional_value.html" title="Fish, surimi">Fish, surimi</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15109" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Frog_legs%2C_raw_nutritional_value.html" title="Frog legs, raw">Frog legs, raw</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=80200" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_ling_nutritional_value.html" title="Fish, raw, ling">Fish, raw, ling</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15044" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_cusk_nutritional_value.html" title="Fish, raw, cusk">Fish, raw, cusk</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15022" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_carp_nutritional_value.html" title="Fish, raw, carp">Fish, raw, carp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15008" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_spot_nutritional_value.html" title="Fish, raw, spot">Fish, raw, spot</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15103" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_scup_nutritional_value.html" title="Fish, raw, scup">Fish, raw, scup</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15090" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_tuna_salad_nutritional_value.html" title="Fish, tuna salad">Fish, tuna salad</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15128" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_cisco_nutritional_value.html" title="Fish, raw, cisco">Fish, raw, cisco</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15013" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_burbot_nutritional_value.html" title="Fish, raw, burbot">Fish, raw, burbot</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15006" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_lingcod_nutritional_value.html" title="Fish, raw, lingcod">Fish, raw, lingcod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15045" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_haddock_nutritional_value.html" title="Fish, raw, haddock">Fish, raw, haddock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15033" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Turtle%2C_raw%2C_green_nutritional_value.html" title="Turtle, raw, green">Turtle, raw, green</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=93600" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_tilapia_nutritional_value.html" title="Fish, raw, tilapia">Fish, raw, tilapia</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15261" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_tilefish_nutritional_value.html" title="Fish, raw, tilefish">Fish, raw, tilefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15112" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mahimahi_nutritional_value.html" title="Fish, raw, mahimahi">Fish, raw, mahimahi</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15023" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_bluefish_nutritional_value.html" title="Fish, raw, bluefish">Fish, raw, bluefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15005" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_monkfish_nutritional_value.html" title="Fish, raw, monkfish">Fish, raw, monkfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15054" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_milkfish_nutritional_value.html" title="Fish, raw, milkfish">Fish, raw, milkfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15053" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_smoked%2C_cisco_nutritional_value.html" title="Fish, smoked, cisco">Fish, smoked, cisco</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15014" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_swordfish_nutritional_value.html" title="Fish, raw, swordfish">Fish, raw, swordfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15110" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_sablefish_nutritional_value.html" title="Fish, raw, sablefish">Fish, raw, sablefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15074" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_snail_nutritional_value.html" title="Mollusks, raw, snail">Mollusks, raw, snail</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=90560" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_butterfish_nutritional_value.html" title="Fish, raw, butterfish">Fish, raw, butterfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15007" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_smoked%2C_haddock_nutritional_value.html" title="Fish, smoked, haddock">Fish, smoked, haddock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15035" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_sheepshead_nutritional_value.html" title="Fish, raw, sheepshead">Fish, raw, sheepshead</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15097" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_ocean%2C_pout_nutritional_value.html" title="Fish, raw, ocean, pout">Fish, raw, ocean, pout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15059" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_salted%2C_mackerel_nutritional_value.html" title="Fish, salted, mackerel">Fish, salted, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=83110" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_pink%2C_salmon_nutritional_value.html" title="Fish, raw, pink, salmon">Fish, raw, pink, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15083" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_smoked%2C_sablefish_nutritional_value.html" title="Fish, smoked, sablefish">Fish, smoked, sablefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15075" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_chum%2C_salmon_nutritional_value.html" title="Fish, raw, chum, salmon">Fish, raw, chum, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15079" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_walleye%2C_pike_nutritional_value.html" title="Fish, raw, walleye, pike">Fish, raw, walleye, pike</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15064" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Jellyfish%2C_salted%2C_dried_nutritional_value.html" title="Jellyfish, salted, dried">Jellyfish, salted, dried</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=43497" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_striped%2C_bass_nutritional_value.html" title="Fish, raw, striped, bass">Fish, raw, striped, bass</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15004" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_white%2C_sucker_nutritional_value.html" title="Fish, raw, white, sucker">Fish, raw, white, sucker</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15107" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic%2C_cod_nutritional_value.html" title="Fish, raw, Atlantic, cod">Fish, raw, Atlantic, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15015" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_northern%2C_pike_nutritional_value.html" title="Fish, raw, northern, pike">Fish, raw, northern, pike</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15062" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_king%2C_mackerel_nutritional_value.html" title="Fish, raw, king, mackerel">Fish, raw, king, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15049" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_orange%2C_roughy_nutritional_value.html" title="Fish, raw, orange, roughy">Fish, raw, orange, roughy</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15073" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_rainbow%2C_smelt_nutritional_value.html" title="Fish, raw, rainbow, smelt">Fish, raw, rainbow, smelt</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15099" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><th class="left results" colspan="1"><a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=1" title="Page 1" class="nav-current">1</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=2" title="Page 2" class="nav">2</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=3" title="Page 3" class="nav">3</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=4" title="Page 4" class="nav">4</a> ... <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=7" title="Page 7" class="nav">7</a> </th><th class="results right" colspan="2"> <a class="rpp" href="javascript:setNResults(10);void(0);" rel="nofollow" title="10 results per page">10</a> <a class="rpp" href="javascript:setNResults(20);void(0);" rel="nofollow" title="20 results per page">20</a> <a class="rpp-current" href="javascript:setNResults(40);void(0);" rel="nofollow" title="40 results per page">40</a></th></tr></tbody></table></td></tr><tbody><tr class="results"><th class="results">Food Name&nbsp;&nbsp;<span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;descending&quot;)" title="Click to sort in descending order.">▲</span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;ascending&quot;)" title="Click to sort in ascending order.">▼</span></span></th><th class="results">Action</th></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_american%2C_shad_nutritional_value.html" title="Fish, raw, american, shad">Fish, raw, american, shad</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15094" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_sockeye%2C_salmon_nutritional_value.html" title="Fish, raw, sockeye, salmon">Fish, raw, sockeye, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15085" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_chinook%2C_salmon_nutritional_value.html" title="Fish, raw, chinook, salmon">Fish, raw, chinook, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15078" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_striped%2C_mullet_nutritional_value.html" title="Fish, raw, striped, mullet">Fish, raw, striped, mullet</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15055" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_florida%2C_pompano_nutritional_value.html" title="Fish, raw, florida, pompano">Fish, raw, florida, pompano</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15068" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Pacific%2C_herring_nutritional_value.html" title="Fish, raw, Pacific, herring">Fish, raw, Pacific, herring</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15043" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_freshwater%2C_drum_nutritional_value.html" title="Fish, raw, freshwater, drum">Fish, raw, freshwater, drum</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15024" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_european%2C_turbot_nutritional_value.html" title="Fish, raw, european, turbot">Fish, raw, european, turbot</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15129" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_blue%2C_mussel_nutritional_value.html" title="Mollusks, raw, blue, mussel">Mollusks, raw, blue, mussel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15164" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_blue%2C_crab_nutritional_value.html" title="Crustaceans, raw, blue, crab">Crustaceans, raw, blue, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15139" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic%2C_pollock_nutritional_value.html" title="Fish, raw, Atlantic, pollock">Fish, raw, Atlantic, pollock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15065" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic%2C_croaker_nutritional_value.html" title="Fish, raw, Atlantic, croaker">Fish, raw, Atlantic, croaker</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15020" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_european%2C_anchovy_nutritional_value.html" title="Fish, raw, european, anchovy">Fish, raw, european, anchovy</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15001" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_carp_nutritional_value.html" title="Fish, dry heat, cooked, carp">Fish, dry heat, cooked, carp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15009" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_spot_nutritional_value.html" title="Fish, dry heat, cooked, spot">Fish, dry heat, cooked, spot</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15216" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_scup_nutritional_value.html" title="Fish, dry heat, cooked, scup">Fish, dry heat, cooked, scup</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15213" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_spanish%2C_mackerel_nutritional_value.html" title="Fish, raw, spanish, mackerel">Fish, raw, spanish, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15051" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic%2C_herring_nutritional_value.html" title="Fish, raw, Atlantic, herring">Fish, raw, Atlantic, herring</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15039" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_cusk_nutritional_value.html" title="Fish, dry heat, cooked, cusk">Fish, dry heat, cooked, cusk</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15193" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_ling_nutritional_value.html" title="Fish, dry heat, cooked, ling">Fish, dry heat, cooked, ling</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15198" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic%2C_wolffish_nutritional_value.html" title="Fish, raw, Atlantic, wolffish">Fish, raw, Atlantic, wolffish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15134" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic%2C_mackerel_nutritional_value.html" title="Fish, raw, Atlantic, mackerel">Fish, raw, Atlantic, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15046" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_eel_nutritional_value.html" title="Fish, raw, mixed species, eel">Fish, raw, mixed species, eel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15025" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_wild%2C_coho%2C_salmon_nutritional_value.html" title="Fish, raw, wild, coho, salmon">Fish, raw, wild, coho, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15081" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_queen%2C_crab_nutritional_value.html" title="Crustaceans, raw, queen, crab">Crustaceans, raw, queen, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15144" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_smoked%2C_chinook%2C_salmon_nutritional_value.html" title="Fish, smoked, chinook, salmon">Fish, smoked, chinook, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15077" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_roe_nutritional_value.html" title="Fish, raw, mixed species, roe">Fish, raw, mixed species, roe</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15072" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Greenland%2C_halibut_nutritional_value.html" title="Fish, raw, Greenland, halibut">Fish, raw, Greenland, halibut</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15038" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_Pacific%2C_oyster_nutritional_value.html" title="Mollusks, raw, Pacific, oyster">Mollusks, raw, Pacific, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15171" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_common%2C_octopus_nutritional_value.html" title="Mollusks, raw, common, octopus">Mollusks, raw, common, octopus</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15166" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_burbot_nutritional_value.html" title="Fish, dry heat, cooked, burbot">Fish, dry heat, cooked, burbot</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15190" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_wild%2C_rainbow%2C_trout_nutritional_value.html" title="Fish, raw, wild, rainbow, trout">Fish, raw, wild, rainbow, trout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15115" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_trout_nutritional_value.html" title="Fish, raw, mixed species, trout">Fish, raw, mixed species, trout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15114" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_bluefin%2C_fresh%2C_tuna_nutritional_value.html" title="Fish, raw, bluefin, fresh, tuna">Fish, raw, bluefin, fresh, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15117" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_perch_nutritional_value.html" title="Fish, raw, mixed species, perch">Fish, raw, mixed species, perch</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15060" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_farmed%2C_coho%2C_salmon_nutritional_value.html" title="Fish, raw, farmed, coho, salmon">Fish, raw, farmed, coho, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15238" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_canned%2C_blue%2C_crab_nutritional_value.html" title="Crustaceans, canned, blue, crab">Crustaceans, canned, blue, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15141" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_haddock_nutritional_value.html" title="Fish, dry heat, cooked, haddock">Fish, dry heat, cooked, haddock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15034" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_lingcod_nutritional_value.html" title="Fish, dry heat, cooked, lingcod">Fish, dry heat, cooked, lingcod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15199" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_tilapia_nutritional_value.html" title="Fish, dry heat, cooked, tilapia">Fish, dry heat, cooked, tilapia</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15262" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><th class="left results" colspan="1"><a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=1" title="Page 1" class="nav">1</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=2" title="Page 2" class="nav-current">2</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=3" title="Page 3" class="nav">3</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=4" title="Page 4" class="nav">4</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=5" title="Page 5" class="nav">5</a> ... <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=7" title="Page 7" class="nav">7</a> </th><th class="results right" colspan="2"> <a class="rpp" href="javascript:setNResults(10);void(0);" rel="nofollow" title="10 results per page">10</a> <a class="rpp" href="javascript:setNResults(20);void(0);" rel="nofollow" title="20 results per page">20</a> <a class="rpp-current" href="javascript:setNResults(40);void(0);" rel="nofollow" title="40 results per page">40</a></th></tr></tbody><tbody><tr class="results"><th class="results">Food Name&nbsp;&nbsp;<span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;descending&quot;)" title="Click to sort in descending order.">▲</span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;ascending&quot;)" title="Click to sort in ascending order.">▼</span></span></th><th class="results">Action</th></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_shark_nutritional_value.html" title="Fish, raw, mixed species, shark">Fish, raw, mixed species, shark</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15095" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_tilefish_nutritional_value.html" title="Fish, dry heat, cooked, tilefish">Fish, dry heat, cooked, tilefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15113" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_pickled%2C_Atlantic%2C_herring_nutritional_value.html" title="Fish, pickled, Atlantic, herring">Fish, pickled, Atlantic, herring</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15041" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_bluefish_nutritional_value.html" title="Fish, dry heat, cooked, bluefish">Fish, dry heat, cooked, bluefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15189" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_pumpkin_seed%2C_sunfish_nutritional_value.html" title="Fish, raw, pumpkin seed, sunfish">Fish, raw, pumpkin seed, sunfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15108" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_skipjack%2C_fresh%2C_tuna_nutritional_value.html" title="Fish, raw, skipjack, fresh, tuna">Fish, raw, skipjack, fresh, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15123" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic%2C_ocean_perch_nutritional_value.html" title="Fish, raw, Atlantic, ocean perch">Fish, raw, Atlantic, ocean perch</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15057" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_monkfish_nutritional_value.html" title="Fish, dry heat, cooked, monkfish">Fish, dry heat, cooked, monkfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15203" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_milkfish_nutritional_value.html" title="Fish, dry heat, cooked, milkfish">Fish, dry heat, cooked, milkfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15202" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mahimahi_nutritional_value.html" title="Fish, dry heat, cooked, mahimahi">Fish, dry heat, cooked, mahimahi</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15194" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_swordfish_nutritional_value.html" title="Fish, dry heat, cooked, swordfish">Fish, dry heat, cooked, swordfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15111" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_whiting_nutritional_value.html" title="Fish, raw, mixed species, whiting">Fish, raw, mixed species, whiting</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15132" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_unspecified%2C_whelk_nutritional_value.html" title="Mollusks, raw, unspecified, whelk">Mollusks, raw, unspecified, whelk</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15177" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_kippered%2C_Atlantic%2C_herring_nutritional_value.html" title="Fish, kippered, Atlantic, herring">Fish, kippered, Atlantic, herring</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15042" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_baked_or_broiled%2C_conch_nutritional_value.html" title="Mollusks, baked or broiled, conch">Mollusks, baked or broiled, conch</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15250" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_snapper_nutritional_value.html" title="Fish, raw, mixed species, snapper">Fish, raw, mixed species, snapper</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15101" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_yellowfin%2C_fresh%2C_tuna_nutritional_value.html" title="Fish, raw, yellowfin, fresh, tuna">Fish, raw, yellowfin, fresh, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15127" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_dungeness%2C_crab_nutritional_value.html" title="Crustaceans, raw, dungeness, crab">Crustaceans, raw, dungeness, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15143" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_wild%2C_Atlantic%2C_salmon_nutritional_value.html" title="Fish, raw, wild, Atlantic, salmon">Fish, raw, wild, Atlantic, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15076" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_grouper_nutritional_value.html" title="Fish, raw, mixed species, grouper">Fish, raw, mixed species, grouper</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15031" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_wild%2C_channel%2C_catfish_nutritional_value.html" title="Fish, raw, wild, channel, catfish">Fish, raw, wild, channel, catfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15010" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_sablefish_nutritional_value.html" title="Fish, dry heat, cooked, sablefish">Fish, dry heat, cooked, sablefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15208" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_canned%2C_eastern%2C_oyster_nutritional_value.html" title="Mollusks, canned, eastern, oyster">Mollusks, canned, eastern, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15170" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_farmed%2C_rainbow%2C_trout_nutritional_value.html" title="Fish, raw, farmed, rainbow, trout">Fish, raw, farmed, rainbow, trout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15240" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_mixed_species%2C_clam_nutritional_value.html" title="Mollusks, raw, mixed species, clam">Mollusks, raw, mixed species, clam</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15157" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_sturgeon_nutritional_value.html" title="Fish, raw, mixed species, sturgeon">Fish, raw, mixed species, sturgeon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15104" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_butterfish_nutritional_value.html" title="Fish, dry heat, cooked, butterfish">Fish, dry heat, cooked, butterfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15191" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_seatrout_nutritional_value.html" title="Fish, raw, mixed species, seatrout">Fish, raw, mixed species, seatrout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15093" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_sheepshead_nutritional_value.html" title="Fish, dry heat, cooked, sheepshead">Fish, dry heat, cooked, sheepshead</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15098" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_sea_bass_nutritional_value.html" title="Fish, raw, mixed species, sea bass">Fish, raw, mixed species, sea bass</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15091" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_whitefish_nutritional_value.html" title="Fish, raw, mixed species, whitefish">Fish, raw, mixed species, whitefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15130" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_alaska_king%2C_crab_nutritional_value.html" title="Crustaceans, raw, alaska king, crab">Crustaceans, raw, alaska king, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15136" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_mixed_species%2C_squid_nutritional_value.html" title="Mollusks, raw, mixed species, squid">Mollusks, raw, mixed species, squid</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15175" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_prepared%2C_frozen%2C_fish_sticks_nutritional_value.html" title="Fish, prepared, frozen, fish sticks">Fish, prepared, frozen, fish sticks</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15027" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_farmed%2C_channel%2C_catfish_nutritional_value.html" title="Fish, raw, farmed, channel, catfish">Fish, raw, farmed, channel, catfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15234" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_farmed%2C_Atlantic%2C_salmon_nutritional_value.html" title="Fish, raw, farmed, Atlantic, salmon">Fish, raw, farmed, Atlantic, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15236" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_northern%2C_lobster_nutritional_value.html" title="Crustaceans, raw, northern, lobster">Crustaceans, raw, northern, lobster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15147" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_ocean%2C_pout_nutritional_value.html" title="Fish, dry heat, cooked, ocean, pout">Fish, dry heat, cooked, ocean, pout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15206" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_yellowtail_nutritional_value.html" title="Fish, raw, mixed species, yellowtail">Fish, raw, mixed species, yellowtail</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15135" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_pink%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, pink, salmon">Fish, dry heat, cooked, pink, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15212" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><th class="left results" colspan="1"><a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=1" title="Page 1" class="nav">1</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=2" title="Page 2" class="nav">2</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=3" title="Page 3" class="nav-current">3</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=4" title="Page 4" class="nav">4</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=5" title="Page 5" class="nav">5</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=6" title="Page 6" class="nav">6</a> ... <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=7" title="Page 7" class="nav">7</a> </th><th class="results right" colspan="2"> <a class="rpp" href="javascript:setNResults(10);void(0);" rel="nofollow" title="10 results per page">10</a> <a class="rpp" href="javascript:setNResults(20);void(0);" rel="nofollow" title="20 results per page">20</a> <a class="rpp-current" href="javascript:setNResults(40);void(0);" rel="nofollow" title="40 results per page">40</a></th></tr></tbody><tbody><tr class="results"><th class="results">Food Name&nbsp;&nbsp;<span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;descending&quot;)" title="Click to sort in descending order.">▲</span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;ascending&quot;)" title="Click to sort in ascending order.">▼</span></span></th><th class="results">Action</th></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_chum%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, chum, salmon">Fish, dry heat, cooked, chum, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15211" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_wild%2C_eastern%2C_oyster_nutritional_value.html" title="Mollusks, raw, wild, eastern, oyster">Mollusks, raw, wild, eastern, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15167" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_mixed_species%2C_abalone_nutritional_value.html" title="Mollusks, raw, mixed species, abalone">Mollusks, raw, mixed species, abalone</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15155" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_mixed_species%2C_scallop_nutritional_value.html" title="Mollusks, raw, mixed species, scallop">Mollusks, raw, mixed species, scallop</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15172" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_white%2C_sucker_nutritional_value.html" title="Fish, dry heat, cooked, white, sucker">Fish, dry heat, cooked, white, sucker</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15217" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_striped%2C_bass_nutritional_value.html" title="Fish, dry heat, cooked, striped, bass">Fish, dry heat, cooked, striped, bass</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15188" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_smoked%2C_mixed_species%2C_sturgeon_nutritional_value.html" title="Fish, smoked, mixed species, sturgeon">Fish, smoked, mixed species, sturgeon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15106" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_granular%2C_black_and_red%2C_caviar_nutritional_value.html" title="Fish, granular, black and red, caviar">Fish, granular, black and red, caviar</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15012" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Atlantic%2C_cod_nutritional_value.html" title="Fish, dry heat, cooked, Atlantic, cod">Fish, dry heat, cooked, Atlantic, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15016" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dried_and_salted%2C_Atlantic%2C_cod_nutritional_value.html" title="Fish, dried and salted, Atlantic, cod">Fish, dried and salted, Atlantic, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15018" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_walleye%2C_pike_nutritional_value.html" title="Fish, dry heat, cooked, walleye, pike">Fish, dry heat, cooked, walleye, pike</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15204" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_smoked%2C_mixed_species%2C_whitefish_nutritional_value.html" title="Fish, smoked, mixed species, whitefish">Fish, smoked, mixed species, whitefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15131" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_northern%2C_pike_nutritional_value.html" title="Fish, dry heat, cooked, northern, pike">Fish, dry heat, cooked, northern, pike</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15063" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_american%2C_shad_nutritional_value.html" title="Fish, dry heat, cooked, american, shad">Fish, dry heat, cooked, american, shad</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15215" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_orange%2C_roughy_nutritional_value.html" title="Fish, dry heat, cooked, orange, roughy">Fish, dry heat, cooked, orange, roughy</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15232" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_rainbow%2C_smelt_nutritional_value.html" title="Fish, dry heat, cooked, rainbow, smelt">Fish, dry heat, cooked, rainbow, smelt</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15100" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_king%2C_mackerel_nutritional_value.html" title="Fish, dry heat, cooked, king, mackerel">Fish, dry heat, cooked, king, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15200" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_farmed%2C_eastern%2C_oyster_nutritional_value.html" title="Mollusks, raw, farmed, eastern, oyster">Mollusks, raw, farmed, eastern, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15245" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_chinook%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, chinook, salmon">Fish, dry heat, cooked, chinook, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15210" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_sockeye%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, sockeye, salmon">Fish, dry heat, cooked, sockeye, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15086" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_New_York_State%2C_raw%2C_brook%2C_trout_nutritional_value.html" title="Fish, New York State, raw, brook, trout">Fish, New York State, raw, brook, trout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15274" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_striped%2C_mullet_nutritional_value.html" title="Fish, dry heat, cooked, striped, mullet">Fish, dry heat, cooked, striped, mullet</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15056" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_florida%2C_pompano_nutritional_value.html" title="Fish, dry heat, cooked, florida, pompano">Fish, dry heat, cooked, florida, pompano</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15069" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_raw%2C_mixed_species%2C_cuttlefish_nutritional_value.html" title="Mollusks, raw, mixed species, cuttlefish">Mollusks, raw, mixed species, cuttlefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15163" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_Atlantic_and_Pacific%2C_halibut_nutritional_value.html" title="Fish, raw, Atlantic and Pacific, halibut">Fish, raw, Atlantic and Pacific, halibut</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15036" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_freshwater%2C_drum_nutritional_value.html" title="Fish, dry heat, cooked, freshwater, drum">Fish, dry heat, cooked, freshwater, drum</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15195" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Pacific%2C_herring_nutritional_value.html" title="Fish, dry heat, cooked, Pacific, herring">Fish, dry heat, cooked, Pacific, herring</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15197" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_european%2C_turbot_nutritional_value.html" title="Fish, dry heat, cooked, european, turbot">Fish, dry heat, cooked, european, turbot</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15222" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Atlantic%2C_herring_nutritional_value.html" title="Fish, dry heat, cooked, Atlantic, herring">Fish, dry heat, cooked, Atlantic, herring</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15040" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_spanish%2C_mackerel_nutritional_value.html" title="Fish, dry heat, cooked, spanish, mackerel">Fish, dry heat, cooked, spanish, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15052" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Atlantic%2C_pollock_nutritional_value.html" title="Fish, dry heat, cooked, Atlantic, pollock">Fish, dry heat, cooked, Atlantic, pollock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15205" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_canned%2C_mixed_species%2C_shrimp_nutritional_value.html" title="Crustaceans, canned, mixed species, shrimp">Crustaceans, canned, mixed species, shrimp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15152" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Atlantic%2C_mackerel_nutritional_value.html" title="Fish, dry heat, cooked, Atlantic, mackerel">Fish, dry heat, cooked, Atlantic, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15047" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_eel_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, eel">Fish, dry heat, cooked, mixed species, eel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15026" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_moist_heat%2C_cooked%2C_blue%2C_mussel_nutritional_value.html" title="Mollusks, moist heat, cooked, blue, mussel">Mollusks, moist heat, cooked, blue, mussel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15165" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_roe_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, roe">Fish, dry heat, cooked, mixed species, roe</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15207" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_greenland%2C_halibut_nutritional_value.html" title="Fish, dry heat, cooked, greenland, halibut">Fish, dry heat, cooked, greenland, halibut</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15196" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Atlantic%2C_wolffish_nutritional_value.html" title="Fish, dry heat, cooked, Atlantic, wolffish">Fish, dry heat, cooked, Atlantic, wolffish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15224" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_wild%2C_coho%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, wild, coho, salmon">Fish, dry heat, cooked, wild, coho, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15247" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned%2C_pink%2C_salmon_nutritional_value.html" title="Fish, drained solids, canned, pink, salmon">Fish, drained solids, canned, pink, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15260" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><th class="left results" colspan="1"><a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=1" title="Page 1" class="nav">1</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=2" title="Page 2" class="nav">2</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=3" title="Page 3" class="nav">3</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=4" title="Page 4" class="nav-current">4</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=5" title="Page 5" class="nav">5</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=6" title="Page 6" class="nav">6</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=7" title="Page 7" class="nav">7</a> </th><th class="results right" colspan="2"> <a class="rpp" href="javascript:setNResults(10);void(0);" rel="nofollow" title="10 results per page">10</a> <a class="rpp" href="javascript:setNResults(20);void(0);" rel="nofollow" title="20 results per page">20</a> <a class="rpp-current" href="javascript:setNResults(40);void(0);" rel="nofollow" title="40 results per page">40</a></th></tr></tbody><tbody><tr class="results"><th class="results">Food Name&nbsp;&nbsp;<span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;descending&quot;)" title="Click to sort in descending order.">▲</span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;ascending&quot;)" title="Click to sort in ascending order.">▼</span></span></th><th class="results">Action</th></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_fresh_water%2C_bass_nutritional_value.html" title="Fish, raw, mixed species, fresh water, bass">Fish, raw, mixed species, fresh water, bass</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15003" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Salmon%2C_total_can_contents%2C_canned%2C_sockeye_nutritional_value.html" title="Salmon, total can contents, canned, sockeye">Salmon, total can contents, canned, sockeye</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15253" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_blue%2C_crab_nutritional_value.html" title="Crustaceans, moist heat, cooked, blue, crab">Crustaceans, moist heat, cooked, blue, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15140" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_Pacific%2C_rockfish_nutritional_value.html" title="Fish, raw, mixed species, Pacific, rockfish">Fish, raw, mixed species, Pacific, rockfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15070" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_sweet_recipe%2C_commercial%2C_gefiltefish_nutritional_value.html" title="Fish, sweet recipe, commercial, gefiltefish">Fish, sweet recipe, commercial, gefiltefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15030" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_bluefin%2C_fresh%2C_tuna_nutritional_value.html" title="Fish, dry heat, cooked, bluefin, fresh, tuna">Fish, dry heat, cooked, bluefin, fresh, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15118" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_perch_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, perch">Fish, dry heat, cooked, mixed species, perch</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15061" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned%2C_jack%2C_mackerel_nutritional_value.html" title="Fish, drained solids, canned, jack, mackerel">Fish, drained solids, canned, jack, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15048" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_wild%2C_rainbow%2C_trout_nutritional_value.html" title="Fish, dry heat, cooked, wild, rainbow, trout">Fish, dry heat, cooked, wild, rainbow, trout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15116" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_farmed%2C_coho%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, farmed, coho, salmon">Fish, dry heat, cooked, farmed, coho, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15239" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_moist_heat%2C_cooked%2C_wild%2C_coho%2C_salmon_nutritional_value.html" title="Fish, moist heat, cooked, wild, coho, salmon">Fish, moist heat, cooked, wild, coho, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15082" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_trout_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, trout">Fish, dry heat, cooked, mixed species, trout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15219" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_queen%2C_crab_nutritional_value.html" title="Crustaceans, moist heat, cooked, queen, crab">Crustaceans, moist heat, cooked, queen, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15227" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_regular%2C_%28lox%29%2C_smoked%2C_chinook%2C_salmon_nutritional_value.html" title="Fish, regular, (lox), smoked, chinook, salmon">Fish, regular, (lox), smoked, chinook, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15179" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_fried%2C_cooked%2C_mixed_species%2C_squid_nutritional_value.html" title="Mollusks, fried, cooked, mixed species, squid">Mollusks, fried, cooked, mixed species, squid</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15176" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_pumpkin_seed%2C_sunfish_nutritional_value.html" title="Fish, dry heat, cooked, pumpkin seed, sunfish">Fish, dry heat, cooked, pumpkin seed, sunfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15218" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_moist_heat%2C_cooked%2C_common%2C_octopus_nutritional_value.html" title="Mollusks, moist heat, cooked, common, octopus">Mollusks, moist heat, cooked, common, octopus</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15230" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_moist_heat%2C_cooked%2C_Pacific%2C_oyster_nutritional_value.html" title="Mollusks, moist heat, cooked, Pacific, oyster">Mollusks, moist heat, cooked, Pacific, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15231" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned%2C_sockeye%2C_salmon_nutritional_value.html" title="Fish, drained solids, canned, sockeye, salmon">Fish, drained solids, canned, sockeye, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15087" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_liquid%2C_canned%2C_mixed_species%2C_clam_nutritional_value.html" title="Mollusks, liquid, canned, mixed species, clam">Mollusks, liquid, canned, mixed species, clam</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15162" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Atlantic%2C_ocean_perch_nutritional_value.html" title="Fish, dry heat, cooked, Atlantic, ocean perch">Fish, dry heat, cooked, Atlantic, ocean perch</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15058" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_fresh%2C_skipjack%2C_tuna_nutritional_value.html" title="Fish, dry heat, cooked, fresh, skipjack, tuna">Fish, dry heat, cooked, fresh, skipjack, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15220" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_whiting_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, whiting">Fish, dry heat, cooked, mixed species, whiting</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15133" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_mixed_species%2C_spiny_lobster_nutritional_value.html" title="Crustaceans, raw, mixed species, spiny lobster">Crustaceans, raw, mixed species, spiny lobster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15154" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_wild%2C_channel%2C_catfish_nutritional_value.html" title="Fish, dry heat, cooked, wild, channel, catfish">Fish, dry heat, cooked, wild, channel, catfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15233" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_total_can_contents%2C_canned%2C_pink%2C_salmon_nutritional_value.html" title="Fish, total can contents, canned, pink, salmon">Fish, total can contents, canned, pink, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15084" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_snapper_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, snapper">Fish, dry heat, cooked, mixed species, snapper</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15102" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_grouper_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, grouper">Fish, dry heat, cooked, mixed species, grouper</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15032" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_solids_and_liquid%2C_canned%2C_Atlantic%2C_cod_nutritional_value.html" title="Fish, solids and liquid, canned, Atlantic, cod">Fish, solids and liquid, canned, Atlantic, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15017" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_wild%2C_Atlantic%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, wild, Atlantic, salmon">Fish, dry heat, cooked, wild, Atlantic, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15209" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_fresh%2C_yellowfin%2C_tuna_nutritional_value.html" title="Fish, dry heat, cooked, fresh, yellowfin, tuna">Fish, dry heat, cooked, fresh, yellowfin, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15221" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_farmed%2C_rainbow%2C_trout_nutritional_value.html" title="Fish, dry heat, cooked, farmed, rainbow, trout">Fish, dry heat, cooked, farmed, rainbow, trout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15241" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_fried%2C_cooked%2C_mixed_species%2C_abalone_nutritional_value.html" title="Mollusks, fried, cooked, mixed species, abalone">Mollusks, fried, cooked, mixed species, abalone</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15156" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_flatfish_%28flounder_and_sole_species%29_nutritional_value.html" title="Fish, raw, flatfish (flounder and sole species)">Fish, raw, flatfish (flounder and sole species)</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15028" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_seatrout_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, seatrout">Fish, dry heat, cooked, mixed species, seatrout</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15214" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_sturgeon_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, sturgeon">Fish, dry heat, cooked, mixed species, sturgeon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15105" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_wild%2C_mixed_species%2C_crayfish_nutritional_value.html" title="Crustaceans, raw, wild, mixed species, crayfish">Crustaceans, raw, wild, mixed species, crayfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15145" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw_%28not_previously_frozen%29%2C_Pacific%2C_cod_nutritional_value.html" title="Fish, raw (not previously frozen), Pacific, cod">Fish, raw (not previously frozen), Pacific, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15268" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_sea_bass_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, sea bass">Fish, dry heat, cooked, mixed species, sea bass</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15092" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned_in_oil%2C_light%2C_tuna_nutritional_value.html" title="Fish, drained solids, canned in oil, light, tuna">Fish, drained solids, canned in oil, light, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15119" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><th class="left results" colspan="1"><a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=1" title="Page 1" class="nav">1</a> ... <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=2" title="Page 2" class="nav">2</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=3" title="Page 3" class="nav">3</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=4" title="Page 4" class="nav">4</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=5" title="Page 5" class="nav-current">5</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=6" title="Page 6" class="nav">6</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=7" title="Page 7" class="nav">7</a> </th><th class="results right" colspan="2"> <a class="rpp" href="javascript:setNResults(10);void(0);" rel="nofollow" title="10 results per page">10</a> <a class="rpp" href="javascript:setNResults(20);void(0);" rel="nofollow" title="20 results per page">20</a> <a class="rpp-current" href="javascript:setNResults(40);void(0);" rel="nofollow" title="40 results per page">40</a></th></tr></tbody><tbody><tr class="results"><th class="results">Food Name&nbsp;&nbsp;<span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;descending&quot;)" title="Click to sort in descending order.">▲</span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;ascending&quot;)" title="Click to sort in ascending order.">▼</span></span></th><th class="results">Action</th></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_moist_heat%2C_cooked%2C_unspecified%2C_whelk_nutritional_value.html" title="Mollusks, moist heat, cooked, unspecified, whelk">Mollusks, moist heat, cooked, unspecified, whelk</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15178" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_farmed%2C_channel%2C_catfish_nutritional_value.html" title="Fish, dry heat, cooked, farmed, channel, catfish">Fish, dry heat, cooked, farmed, channel, catfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15235" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_farmed%2C_Atlantic%2C_salmon_nutritional_value.html" title="Fish, dry heat, cooked, farmed, Atlantic, salmon">Fish, dry heat, cooked, farmed, Atlantic, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15237" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw_%28not_previously_frozen%29%2C_shrimp_nutritional_value.html" title="Crustaceans, raw (not previously frozen), shrimp">Crustaceans, raw (not previously frozen), shrimp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15270" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned_in_oil%2C_white%2C_tuna_nutritional_value.html" title="Fish, drained solids, canned in oil, white, tuna">Fish, drained solids, canned in oil, white, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15124" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_home_recipe%2C_crab_cakes%2C_blue%2C_crab_nutritional_value.html" title="Crustaceans, home recipe, crab cakes, blue, crab">Crustaceans, home recipe, crab cakes, blue, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15142" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_whitefish_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, whitefish">Fish, dry heat, cooked, mixed species, whitefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15223" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_dungeness%2C_crab_nutritional_value.html" title="Crustaceans, moist heat, cooked, dungeness, crab">Crustaceans, moist heat, cooked, dungeness, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15226" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_moist_heat%2C_cooked%2C_mixed_species%2C_clam_nutritional_value.html" title="Mollusks, moist heat, cooked, mixed species, clam">Mollusks, moist heat, cooked, mixed species, clam</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15159" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_steamed%2C_cooked%2C_%28bay_and_sea%29%2C_scallop_nutritional_value.html" title="Mollusks, steamed, cooked, (bay and sea), scallop">Mollusks, steamed, cooked, (bay and sea), scallop</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=90240" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_breaded_and_fried%2C_cooked%2C_channel%2C_catfish_nutritional_value.html" title="Fish, breaded and fried, cooked, channel, catfish">Fish, breaded and fried, cooked, channel, catfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15011" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_yellowtail_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, yellowtail">Fish, dry heat, cooked, mixed species, yellowtail</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15225" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw%2C_farmed%2C_mixed_species%2C_crayfish_nutritional_value.html" title="Crustaceans, raw, farmed, mixed species, crayfish">Crustaceans, raw, farmed, mixed species, crayfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15242" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_dry_heat%2C_cooked%2C_wild%2C_eastern%2C_oyster_nutritional_value.html" title="Mollusks, dry heat, cooked, wild, eastern, oyster">Mollusks, dry heat, cooked, wild, eastern, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15244" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_alaska_king%2C_crab_nutritional_value.html" title="Crustaceans, moist heat, cooked, alaska king, crab">Crustaceans, moist heat, cooked, alaska king, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15137" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_breaded_and_fried%2C_cooked%2C_Atlantic%2C_croaker_nutritional_value.html" title="Fish, breaded and fried, cooked, Atlantic, croaker">Fish, breaded and fried, cooked, Atlantic, croaker</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15021" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned_in_water%2C_white%2C_tuna_nutritional_value.html" title="Fish, drained solids, canned in water, white, tuna">Fish, drained solids, canned in water, white, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15126" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned_in_water%2C_light%2C_tuna_nutritional_value.html" title="Fish, drained solids, canned in water, light, tuna">Fish, drained solids, canned in water, light, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15121" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_northern%2C_lobster_nutritional_value.html" title="Crustaceans, moist heat, cooked, northern, lobster">Crustaceans, moist heat, cooked, northern, lobster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15148" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw_%28not_previously_frozen%29%2C_Alaska%2C_pollock_nutritional_value.html" title="Fish, raw (not previously frozen), Alaska, pollock">Fish, raw (not previously frozen), Alaska, pollock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15266" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_cooked_%28not_previously_frozen%29%2C_Pacific%2C_cod_nutritional_value.html" title="Fish, cooked (not previously frozen), Pacific, cod">Fish, cooked (not previously frozen), Pacific, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15269" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_cooked_%28not_previously_frozen%29%2C_shrimp_nutritional_value.html" title="Crustaceans, cooked (not previously frozen), shrimp">Crustaceans, cooked (not previously frozen), shrimp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15271" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_moist_heat%2C_cooked%2C_wild%2C_eastern%2C_oyster_nutritional_value.html" title="Mollusks, moist heat, cooked, wild, eastern, oyster">Mollusks, moist heat, cooked, wild, eastern, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15169" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_dry_heat%2C_cooked%2C_farmed%2C_eastern%2C_oyster_nutritional_value.html" title="Mollusks, dry heat, cooked, farmed, eastern, oyster">Mollusks, dry heat, cooked, farmed, eastern, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15246" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids_with_bone%2C_canned%2C_chum%2C_salmon_nutritional_value.html" title="Fish, drained solids with bone, canned, chum, salmon">Fish, drained solids with bone, canned, chum, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15080" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_breaded_and_fried%2C_cooked%2C_eastern%2C_oyster_nutritional_value.html" title="Mollusks, breaded and fried, cooked, eastern, oyster">Mollusks, breaded and fried, cooked, eastern, oyster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15168" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw%2C_mixed_species%2C_Pacific_and_jack%2C_mackerel_nutritional_value.html" title="Fish, raw, mixed species, Pacific and jack, mackerel">Fish, raw, mixed species, Pacific and jack, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15050" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_drained_solids%2C_canned%2C_mixed_species%2C_clam_nutritional_value.html" title="Mollusks, drained solids, canned, mixed species, clam">Mollusks, drained solids, canned, mixed species, clam</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15160" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_Atlantic_and_Pacific%2C_halibut_nutritional_value.html" title="Fish, dry heat, cooked, Atlantic and Pacific, halibut">Fish, dry heat, cooked, Atlantic and Pacific, halibut</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15037" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_cooked_%28not_previously_frozen%29%2C_Alaska%2C_pollock_nutritional_value.html" title="Fish, cooked (not previously frozen), Alaska, pollock">Fish, cooked (not previously frozen), Alaska, pollock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15267" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_canned_in_oil%2C_european%2C_anchovy_nutritional_value.html" title="Fish, drained solids, canned in oil, european, anchovy">Fish, drained solids, canned in oil, european, anchovy</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15002" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_freshwater%2C_bass_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, freshwater, bass">Fish, dry heat, cooked, mixed species, freshwater, bass</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15187" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/USDA_Commodity%2C_heated%2C_frozen%2C_breaded%2C_salmon_nuggets_nutritional_value.html" title="USDA Commodity, heated, frozen, breaded, salmon nuggets">USDA Commodity, heated, frozen, breaded, salmon nuggets</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15251" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_moist_heat%2C_cooked%2C_mixed_species%2C_cuttlefish_nutritional_value.html" title="Mollusks, moist heat, cooked, mixed species, cuttlefish">Mollusks, moist heat, cooked, mixed species, cuttlefish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15229" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_breaded_and_fried%2C_cooked%2C_mixed_species%2C_clam_nutritional_value.html" title="Mollusks, breaded and fried, cooked, mixed species, clam">Mollusks, breaded and fried, cooked, mixed species, clam</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15158" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_Pacific%2C_rockfish_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, Pacific, rockfish">Fish, dry heat, cooked, mixed species, Pacific, rockfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15071" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw_%28may_have_been_previously_frozen%29%2C_Pacific%2C_cod_nutritional_value.html" title="Fish, raw (may have been previously frozen), Pacific, cod">Fish, raw (may have been previously frozen), Pacific, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15019" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_made_from_surimi%2C_imitation%2C_alaska_king%2C_crab_nutritional_value.html" title="Crustaceans, made from surimi, imitation, alaska king, crab">Crustaceans, made from surimi, imitation, alaska king, crab</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15138" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_breaded_and_fried%2C_cooked%2C_mixed_species%2C_scallop_nutritional_value.html" title="Mollusks, breaded and fried, cooked, mixed species, scallop">Mollusks, breaded and fried, cooked, mixed species, scallop</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15173" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_batter-dipped_and_fried%2C_cooked%2C_mixed_species%2C_shark_nutritional_value.html" title="Fish, batter-dipped and fried, cooked, mixed species, shark">Fish, batter-dipped and fried, cooked, mixed species, shark</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15096" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><th class="left results" colspan="1"><a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=1" title="Page 1" class="nav">1</a> ... <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=3" title="Page 3" class="nav">3</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=4" title="Page 4" class="nav">4</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=5" title="Page 5" class="nav">5</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=6" title="Page 6" class="nav-current">6</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=7" title="Page 7" class="nav">7</a> </th><th class="results right" colspan="2"> <a class="rpp" href="javascript:setNResults(10);void(0);" rel="nofollow" title="10 results per page">10</a> <a class="rpp" href="javascript:setNResults(20);void(0);" rel="nofollow" title="20 results per page">20</a> <a class="rpp-current" href="javascript:setNResults(40);void(0);" rel="nofollow" title="40 results per page">40</a></th></tr></tbody><tbody><tr class="results"><th class="results">Food Name&nbsp;&nbsp;<span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;descending&quot;)" title="Click to sort in descending order.">▲</span><span class="sort" onclick="changeSortMode(&quot;name&quot;, &quot;ascending&quot;)" title="Click to sort in ascending order.">▼</span></span></th><th class="results">Action</th></tr><tr class="results"><td class="results left"><a href="/Fish%2C_raw_%28may_have_been_previously_frozen%29%2C_Alaska%2C_pollock_nutritional_value.html" title="Fish, raw (may have been previously frozen), Alaska, pollock">Fish, raw (may have been previously frozen), Alaska, pollock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15066" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_flatfish_%28flounder_and_sole_species%29_nutritional_value.html" title="Fish, dry heat, cooked, flatfish (flounder and sole species)">Fish, dry heat, cooked, flatfish (flounder and sole species)</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15029" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_breaded_and_fried%2C_cooked%2C_mixed_species%2C_shrimp_nutritional_value.html" title="Crustaceans, breaded and fried, cooked, mixed species, shrimp">Crustaceans, breaded and fried, cooked, mixed species, shrimp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15150" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Mollusks%2C_made_from_surimi%2C_imitation%2C_mixed_species%2C_scallop_nutritional_value.html" title="Mollusks, made from surimi, imitation, mixed species, scallop">Mollusks, made from surimi, imitation, mixed species, scallop</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15174" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/USDA_Commodity%2C_unheated%2C_cooked_as_purchased%2C_salmon_nuggets_nutritional_value.html" title="USDA Commodity, unheated, cooked as purchased, salmon nuggets">USDA Commodity, unheated, cooked as purchased, salmon nuggets</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15252" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_mixed_species%2C_spiny_lobster_nutritional_value.html" title="Crustaceans, moist heat, cooked, mixed species, spiny lobster">Crustaceans, moist heat, cooked, mixed species, spiny lobster</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15228" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_without_salt%2C_canned_in_oil%2C_white%2C_tuna_nutritional_value.html" title="Fish, drained solids, without salt, canned in oil, white, tuna">Fish, drained solids, without salt, canned in oil, white, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15185" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_without_salt%2C_canned_in_oil%2C_light%2C_tuna_nutritional_value.html" title="Fish, drained solids, without salt, canned in oil, light, tuna">Fish, drained solids, without salt, canned in oil, light, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15183" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_wild%2C_mixed_species%2C_crayfish_nutritional_value.html" title="Crustaceans, moist heat, cooked, wild, mixed species, crayfish">Crustaceans, moist heat, cooked, wild, mixed species, crayfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15146" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_made_from_surimi%2C_imitation%2C_mixed_species%2C_shrimp_nutritional_value.html" title="Crustaceans, made from surimi, imitation, mixed species, shrimp">Crustaceans, made from surimi, imitation, mixed species, shrimp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15153" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Salmon%2C_without_skin_and_bones%2C_drained_solids%2C_canned%2C_sockeye_nutritional_value.html" title="Salmon, without skin and bones, drained solids, canned, sockeye">Salmon, without skin and bones, drained solids, canned, sockeye</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15264" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_without_salt%2C_canned_in_water%2C_white%2C_tuna_nutritional_value.html" title="Fish, drained solids, without salt, canned in water, white, tuna">Fish, drained solids, without salt, canned in water, white, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15186" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids%2C_without_salt%2C_canned_in_water%2C_light%2C_tuna_nutritional_value.html" title="Fish, drained solids, without salt, canned in water, light, tuna">Fish, drained solids, without salt, canned in water, light, tuna</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15184" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids_with_bone%2C_canned_in_oil%2C_Atlantic%2C_sardine_nutritional_value.html" title="Fish, drained solids with bone, canned in oil, Atlantic, sardine">Fish, drained solids with bone, canned in oil, Atlantic, sardine</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15088" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat%2C_cooked%2C_farmed%2C_mixed_species%2C_crayfish_nutritional_value.html" title="Crustaceans, moist heat, cooked, farmed, mixed species, crayfish">Crustaceans, moist heat, cooked, farmed, mixed species, crayfish</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15243" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat%2C_cooked%2C_mixed_species%2C_Pacific_and_jack%2C_mackerel_nutritional_value.html" title="Fish, dry heat, cooked, mixed species, Pacific and jack, mackerel">Fish, dry heat, cooked, mixed species, Pacific and jack, mackerel</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15201" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids_with_bone%2C_without_salt%2C_canned%2C_chum%2C_salmon_nutritional_value.html" title="Fish, drained solids with bone, without salt, canned, chum, salmon">Fish, drained solids with bone, without salt, canned, chum, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15180" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_without_skin_and_bones%2C_drained_solids%2C_canned%2C_pink%2C_Salmon_nutritional_value.html" title="Fish, without skin and bones, drained solids, canned, pink, Salmon">Fish, without skin and bones, drained solids, canned, pink, Salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15265" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids_with_bone%2C_without_salt%2C_canned%2C_sockeye%2C_salmon_nutritional_value.html" title="Fish, drained solids with bone, without salt, canned, sockeye, salmon">Fish, drained solids with bone, without salt, canned, sockeye, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15182" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_solids_with_bone_and_liquid%2C_without_salt%2C_canned%2C_pink%2C_salmon_nutritional_value.html" title="Fish, solids with bone and liquid, without salt, canned, pink, salmon">Fish, solids with bone and liquid, without salt, canned, pink, salmon</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15181" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat_%28may_have_been_previously_frozen%29%2C_cooked%2C_Pacific%2C_cod_nutritional_value.html" title="Fish, dry heat (may have been previously frozen), cooked, Pacific, cod">Fish, dry heat (may have been previously frozen), cooked, Pacific, cod</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15192" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_drained_solids_with_bone%2C_canned_in_tomato_sauce%2C_Pacific%2C_sardine_nutritional_value.html" title="Fish, drained solids with bone, canned in tomato sauce, Pacific, sardine">Fish, drained solids with bone, canned in tomato sauce, Pacific, sardine</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15089" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Fish%2C_dry_heat_%28may_have_been_previously_frozen%29%2C_cooked%2C_Alaska%2C_pollock_nutritional_value.html" title="Fish, dry heat (may have been previously frozen), cooked, Alaska, pollock">Fish, dry heat (may have been previously frozen), cooked, Alaska, pollock</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15067" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_raw_%28may_have_been_previously_frozen%29%2C_mixed_species%2C_shrimp_nutritional_value.html" title="Crustaceans, raw (may have been previously frozen), mixed species, shrimp">Crustaceans, raw (may have been previously frozen), mixed species, shrimp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15149" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><td class="results left"><a href="/Crustaceans%2C_moist_heat_%28may_have_been_previously_frozen%29%2C_cooked%2C_mixed_species%2C_shrimp_nutritional_value.html" title="Crustaceans, moist heat (may have been previously frozen), cooked, mixed species, shrimp">Crustaceans, moist heat (may have been previously frozen), cooked, mixed species, shrimp</a> </td> <td class="results"><a href="/nutritioncalculator.php?action=add&amp;id=15151" rel="nofollow">Add&nbsp;to&nbsp;meal</a></td></tr><tr class="results"><th class="left results" colspan="1"><a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=1" title="Page 1" class="nav">1</a> ... <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=4" title="Page 4" class="nav">4</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=5" title="Page 5" class="nav">5</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=6" title="Page 6" class="nav">6</a> <a href="/search.php?category=Finfish+and+Shellfish+Products&amp;page=7" title="Page 7" class="nav-current">7</a> </th><th class="results right" colspan="2"> <a class="rpp" href="javascript:setNResults(10);void(0);" rel="nofollow" title="10 results per page">10</a> <a class="rpp" href="javascript:setNResults(20);void(0);" rel="nofollow" title="20 results per page">20</a> <a class="rpp-current" href="javascript:setNResults(40);void(0);" rel="nofollow" title="40 results per page">40</a></th></tr></tbody>'

In [25]:
def getURL(page): # this block is not mine in origin
    start_link = page.find("a href")
    if start_link == -1:
        return None, 0
    start_quote = page.find('"', start_link)
    end_quote = page.find('"', start_quote + 1)
    url = page[start_quote + 1: end_quote]
    return url, end_quote

url_list_raw = []
while True:
    url, n = getURL(page)
    page = page[n:]
    if url:
        url_list_raw.append(url)
    else:
        break

In [26]:
url_list_final = []
for i in url_list_raw:
    if i.endswith('.html'):
        url_list_final.append('https://www.nutritionvalue.org'+i)
print(url_list_final)

['https://www.nutritionvalue.org/Fish%2C_surimi_nutritional_value.html', 'https://www.nutritionvalue.org/Frog_legs%2C_raw_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_ling_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_cusk_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_carp_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_spot_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_scup_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_tuna_salad_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_cisco_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_burbot_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_lingcod_nutritional_value.html', 'https://www.nutritionvalue.org/Fish%2C_raw%2C_haddock_nutritional_value.html', 'https://www.nutritionvalue.org/Turtle%2C_raw%2C_green_nutritional_value.ht

In [6]:
import csv #saving url_list_final in case I need it later

with open("url_list_final.csv", "w") as f:
    writer = csv.writer(f)
    for i in url_list_final:
        writer.writerow([i])

# Scraping final URLs

In [148]:
url_list_sec1 = url_list_final[:49]
url_list_sec2 = url_list_final[50:99]
url_list_sec3 = url_list_final[100:125]
url_list_sec10 = url_list_final[126:135]
url_list_sec12 = url_list_final[136:139]
#140 is bad
url_list_sec11 = url_list_final[141:149]
url_list_sec4 = url_list_final[150:174]
url_list_sec8 = url_list_final[175:195]
#195-200 is bad
url_list_sec9 = url_list_final[205:215]
#204 is bad, so is 215 to 224
url_list_sec5 = url_list_final[224:229]
#230-232 are bad, so is 245
url_list_sec6 = url_list_final[233:244]
url_list_sec7 = url_list_final[246:265]

In [149]:
function_test_results = test_scrape(url_list_sec12)

In [150]:
function_test_results

[(46.8,
  0.465,
  0.589,
  0.979,
  558.0,
  0.0,
  17.0,
  3.0,
  4.0,
  13.0,
  9.0,
  52.0,
  'Fish, dry heat, cooked, king, mackerel'),
 (63.7,
  0.443,
  0.591,
  0.152,
  124.0,
  0.0,
  0.0,
  8.0,
  4.0,
  32.0,
  8.0,
  10.0,
  'Mollusks, raw, farmed, eastern, oyster'),
 (46.8,
  3.214,
  2.662,
  5.742,
  505.0,
  0.0,
  10.0,
  7.0,
  3.0,
  5.0,
  3.0,
  52.0,
  'Fish, dry heat, cooked, chinook, salmon')]

In [151]:
df_12 = pd.DataFrame.from_records(function_test_results, columns = labels)


In [152]:
df_12.to_csv('sec12_fishes.csv', index=False)